In [1]:
import os
os.chdir("../")

In [ ]:
#Extract data from pdf file

def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    # Tag each document with the PDF filename it came from
    for doc in documents:
        file_path = doc.metadata.get("source", "")
        doc.metadata["source"] = os.path.basename(file_path)

    return documents

In [ ]:
all_docs = load_pdf_file("Data/")  # folder containing book1.pdf and book2.pdf

sources = set([doc.metadata["source"] for doc in all_docs])
print("Books loaded:", sources)


Books loaded: {'Medical_book5.pdf', 'Medical_book4.pdf', 'Medical_book2.pdf', 'Medical_book1.pdf', 'Medical_book3.pdf'}


In [ ]:
import os
print(os.getcwd())

#extracted_data = load_pdf_file(data="c:/Users/NEETHEESWARAN/Git2/Medical-chatbot-Gen-AI/Data/")

extracted_data = load_pdf_file(data="data/")

c:\Users\NEETHEESWARAN\Git2\Medibot-demo


In [ ]:
#Split the data into smaller chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print(f"Total number of chunks: {len(text_chunks)}")

Total number of chunks: 38122


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
#Download embeddings from HuggingFace
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
query_result = embeddings.embed_query("Hi")
print("length of query result:", len(query_result))

length of query result: 384


In [ ]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()  # By default looks for .env in the current directory

# Now retrieve the key
api_key = os.environ.get('PINECONE_API_KEY')
#openai_api_key = os.environ.get('OPENAI_API_KEY')
groq_api_key= os.environ.get('GROQ_API_KEY')

In [ ]:
import pinecone
print(dir(pinecone))

['Admin', 'AwsRegion', 'AzureRegion', 'BackupList', 'BackupModel', 'ByocSpec', 'CloudProvider', 'CollectionDescription', 'CollectionList', 'Config', 'ConfigBuilder', 'ConfigureIndexEmbed', 'CreateIndexForModelEmbedTypedDict', 'DeleteRequest', 'DeletionProtection', 'DescribeIndexStatsRequest', 'DescribeIndexStatsResponse', 'EmbedModel', 'EmbeddingsList', 'FetchResponse', 'ForbiddenException', 'GcpRegion', 'ImportErrorMode', 'IndexEmbed', 'IndexList', 'IndexModel', 'ListConversionException', 'Metric', 'ModelInfo', 'ModelInfoList', 'NamespaceDescription', 'NotFoundException', 'Pinecone', 'PineconeApiAttributeError', 'PineconeApiException', 'PineconeApiKeyError', 'PineconeApiTypeError', 'PineconeApiValueError', 'PineconeAsyncio', 'PineconeConfig', 'PineconeConfigurationError', 'PineconeException', 'PineconeProtocolError', 'PodIndexEnvironment', 'PodSpec', 'PodSpecDefinition', 'PodType', 'QueryRequest', 'QueryResponse', 'RerankModel', 'RerankResult', 'RestoreJobList', 'RestoreJobModel', 'Rp

In [ ]:
#from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone, ServerlessSpec


pc = Pinecone(api_key=api_key)

index_name = "medicalbot"


# pc.create_index(
#     name=index_name,
#     dimension=384, 
#     metric="cosine", 
#     spec=ServerlessSpec(
#         cloud="aws", 
#         region="us-east-1"
#     ) 
# ) 

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)

In [ ]:
import os

os.environ["PINECONE_API_KEY"] = api_key
os.environ["GROQ_API_KEY"] = groq_api_key
#os.environ["OPENAI_API_KEY"] = openai_api_key


In [ ]:
#Embed and upload the data to Pinecone
# If this is your first time adding documents
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

In [ ]:
#Load existing index
from langchain_pinecone import PineconeVectorStore
# Embed each document and upload to the existing index
docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding = embeddings
)


In [ ]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k": 3})

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(temperature=0.7, model_name="llama3-70b-8192") #mixtral-8x7b-32768")  # or 

In [ ]:
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are a knowledgeable and trustworthy AI medical assistant. "
    "Always prioritize answering the user’s question using information from the retrieved medical book or trusted internal knowledge. "
    "If the book content does not contain a clear or complete answer, use available tools to search for up-to-date medical information. "
    "When using external sources, always start with: "
    "'Note: The following answer is based on external information retrieved from [source_name].' "
    "Then provide a clear, accurate, and helpful explanation. "
    "Avoid vague language, and do not mention the word 'context' or reference your training data. "
    "Your responses should be medically sound, professional, and empathetic, written in 4–6 well-structured sentences."
)

    


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# 1. Create memory object
memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True
)

# 2. Create conversational chain with memory
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    verbose=True  # Optional: helpful for debugging
)


C:\Users\NEETHEESWARAN\AppData\Local\Temp\ipykernel_26788\3491887695.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [ ]:
response = qa_chain.invoke({"question": "What is Pneumonia?"})
print(response["answer"])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is Acne?
Assistant: Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
Follow Up Input: What is Pneumonia?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Pneumonia
Definition
Pneumonia is an infection of the lung, and can be
caused by nearly any class of organism known to cause
human infections. These include bacteria, viruses, fungi,
and parasites. In 

In [ ]:
response = invoke_rag_chain_with_retry("What is Acne?")
print(response["answer"] if isinstance(response, dict) else response)
